## Описание проекта

**Matching** - это задача поиска и сопоставления двух объектов из разных наборов данных. 

#### Что надо сделать?

- разработать алгоритм, который для всех товаров из **validation.csv** предложит несколько вариантов наиболее похожих товаров из base;
- оценить качество алгоритма по метрике **accuracy@5**.

 Описание:

***base.csv*** - анонимизированный набор товаров. Каждый товар представлен как уникальный id (0-base, 1-base, 2-base) и вектор признаков размерностью 72.
***train.csv*** - обучающий датасет. Каждая строчка - один товар, для которого известен уникальный id (0-query, 1-query, …), вектор признаков и id товара из base.csv, который максимально похож на него (по мнению экспертов).
***validation.csv*** - датасет с товарами (уникальный id и вектор признаков), для которых надо найти наиболее близкие товары из base.csv
***validation_answer.csv*** - правильные ответы к предыдущему файлу

**Задание:**
- загрузить данные;
- понять задачу;
- подготовить данные;
- обучить базовую модель; 
- измерить качество.

<a id='section_0'></a>
### [Оглавление](#section_0)<br>
[1. Импорты, настройки и загрузка данных](#section_1)<br>
-    [1.1 Функции](#section_1.1)<br>
-    [1.2 Загрузка данных](#section_1.2)<br>

[2. Анализ и предобработка данных](#section_2)<br>
[3. Подготовка и выбор моделей](#section_3)<br>
[4. Уменьшение размерности](#section_4)<br>
[5. Проверка на Валидационной выборкее](#section_5)<br>
[6. Итоговый вывод](#section_6)<br>


<a id='section_1'></a>
## Импорты, настройки и загрузка данных

In [1]:
import faiss
import numpy as np
import pandas as pd
import seaborn as sns
import sweetviz as sv
import time
import warnings

from matplotlib import pyplot as plt
from phik import phik_matrix
from sklearn.decomposition import IncrementalPCA, PCA, TruncatedSVD
from sklearn.cluster import KMeans
from sklearn.metrics import davies_bouldin_score
from sklearn.preprocessing import (MaxAbsScaler, MinMaxScaler, 
                                   PowerTransformer, RobustScaler, 
                                   QuantileTransformer)
from tqdm import tqdm


# константы
RANDOM_STATE = 13
K = 5

In [2]:
# !pip install faiss-cpu -q
# !pip install sweetviz -q
# !pip install phik -q
# !pip install shap -q

In [3]:
# настройки
pd.options.display.float_format = '{:_.4f}'.format
pd.options.display.max_columns = None
warnings.filterwarnings("ignore")

<a id='section_1.1'></a>
### Функции

In [4]:
# Изучение данных
def get_data_info(df): 
    display(df.sample(5))
    display(df.info())
    display(df.describe(include='all'))
    display(f'Количество пропусков в данных: {df.isna().sum().sort_values(ascending=False)}')
    display(f'Кол-во дубликатов в данных = {df.duplicated().sum()}')

# Изменение типа данных
def reduce_mem_usage(df):
    start_mem = df.memory_usage().sum() / 1024**2
    print(f'Memory usage of dataframe is {round(start_mem, 2)} MB')

    cols = df.select_dtypes(exclude=['object']).columns.to_list()
    for col in cols:
        df[col] = df[col].astype(np.float32)

    end_mem = df.memory_usage().sum() / 1024**2
    print(f'Memory usage after optimization is: {round(end_mem, 2)} MB')
    print(f'Decreased by {round(100 * (start_mem - end_mem) / start_mem, 2)}%')

    return df

# Отображение выбросов в данных
def show_outliers(df):
    df.plot(kind='box', figsize=[25, 10]) 
    plt.title('Диаграммa размаха.', fontsize=10)
    plt.tight_layout()
    plt.show();

# Удаление выбросов. 
def removing_outliers(df):
    for i in df.select_dtypes(exclude=['object']).columns.to_list():
        q1 = df[i].quantile(.25)
        q3 = df[i].quantile(.75)
        iqr = q3 - q1
        lower = q1 - 1.5 - iqr
        upper = q3 + 1.5 * iqr
        df[i] = df[i].clip(lower, upper)
    return df

<a id='section_1.2'></a>
### Загрузка данных

In [5]:
try:
    df_train = pd.read_csv('datasets/train.csv')
    df_test = pd.read_csv('datasets/validation.csv')
    df_base = pd.read_csv('datasets/base.csv')
    ans = pd.read_csv('datasets/validation_answer.csv')

except:
    df_train = pd.read_csv('/datasets/train.csv')
    df_test = pd.read_csv('/datasets/validation.csv')
    df_base = pd.read_csv('/datasets/base.csv')
    ans = pd.read_csv('/datasets/validation_answer.csv')

In [6]:
# Посмотрю на полученные данные
for df in [df_train, df_test, df_base]:
    get_data_info(df)

Пропуски и явные дубликаты в данных отсутствуют. 

[К оглавлению](#section_0)
<a id='section_2'></a>
## Анализ и предобработка данных

In [7]:
# Изменю тип данных для уменьшения используемой памяти. 
for df in [df_train, df_test, df_base]:
    reduce_mem_usage(df)

Memory usage of dataframe is 56.46 MB
Memory usage after optimization is: 28.99 MB
Decreased by 48.65%
Memory usage of dataframe is 55.69 MB
Memory usage after optimization is: 28.23 MB
Decreased by 49.31%
Memory usage of dataframe is 1625.25 MB
Memory usage after optimization is: 823.75 MB
Decreased by 49.32%


In [8]:
# Сделаю из колонки с Id индекс. 
for df in [df_base, df_train, df_test]:
    df.set_index('Id', inplace=True)

In [9]:
# Выделю таргет и подготовлю словарь с индексами из базового датасета 
train_targets = df_train.Target
base_index = {k: v for k, v in enumerate(df_base.index.to_list())}

In [10]:
# Проверю дубликаты после удаления колонки с Id 
for df in [df_base, df_train, df_test]:
    display(f'Кол-во дубликатов в данных = {df.duplicated().sum()}')

'Кол-во дубликатов в данных = 0'

'Кол-во дубликатов в данных = 0'

'Кол-во дубликатов в данных = 0'

In [11]:
# Рассмотрим полученные данные. 
for df in [df_base, df_train, df_test]:
    analyze_report = sv.analyze(df)
    analyze_report.show_notebook()
    analyze_report.show_html()

Большинство данных имеют вид нормального или близкого к нормальному распределения. Из общей картины выбиваются колонки №№ 6, 21, 25, 33, 44, 59, 65, 70.

In [12]:
# Посмотрю матрицу корреляции Phik и взаимосвязь признаков с таргетом. 
phik_matrix = df_train.phik_matrix(njobs=-1) 

In [13]:
f, ax = plt.subplots(figsize=(25, 10))
sns.heatmap(phik_matrix)
plt.title('Матрица корреляции $Phi$')
plt.show()

На основе этого графика можно удалить колонки №№ 33 и 59, так как они не оказывают влияния на таргет. 

In [14]:
# Удалю колонку с таргетом из тренировочного датасета
df_train.drop('Target', axis=1, inplace=True)

In [15]:
# Посмотрю на выбросы в данных
show_outliers(df_base)

In [16]:
show_outliers(df_train)

In [17]:
show_outliers(df_test)

Практически у всех признаков имеются выбросы. 
Колонки, на которые ранее обратил внимание, тоже ведут себя на этих графиках отлично от остальных признаков. 

In [18]:
# Метрика с RobustScaler без удаления выбросов показывает лучшие значения, чем с удалением оных. В связи с этим не буду удалять выбросы. 
# for df in [df_train, df_base, df_test]:
#     removing_outliers(df)

In [19]:
# функция вычисления accuracy@n
def accuracy_n(train_df:pd.DataFrame, 
               base_df:pd.DataFrame,
               targets=train_targets,
               base_index_=base_index,
               idx=None):

    accuracy = 0
    start = time.time()
    
    index_ = idx
    index_.reset()
    if type(index_) == faiss.swigfaiss.IndexIVFFlat:
        index_.nprobe = 55
        index_.nlist = 55 
        # при этих показателях у данного индекса максимальное значение метрики
        
    # index_.train(base_df)
    # index_.add(base_df)
    # _, indexes_ = index_.search(train_df, K)
    
    index_.train(np.ascontiguousarray(base_df))
    index_.add(np.ascontiguousarray(base_df))
    _, indexes_ = index_.search(np.ascontiguousarray(train_df), K)

    for target, predicted in zip(targets.tolist(), indexes_.tolist()):
        accuracy += int(target in [base_index_[number] for number in predicted])

    return accuracy / len(indexes_) * 100, round(time.time() - start, 2)

In [20]:
acc, _ = accuracy_n(df_train, df_base, idx=faiss.IndexFlatL2(df_base.shape[1]))
print(f'Метрика базовой модели без воздействий на признаки составляет {acc}')

Метрика базовой модели без воздействий на признаки составляет 13.804


In [21]:
# Посмотрю на изменения показаний метрики при удалении признаков
res_list = []
cols_for_drop = ['6', '21', '25', '33', '59', '44', '65', '70']

df_train_drop = df_train.copy()
df_base_drop = df_base.copy()

for col in tqdm(cols_for_drop):
    df_train_drop.drop(col, axis=1, inplace=True)
    df_base_drop.drop(col, axis=1, inplace=True)

    index = faiss.IndexFlatL2(df_base_drop.shape[1])

    acc, time_ = accuracy_n(df_train_drop, df_base_drop, idx=index)
    res_list.append([col, acc, time_])

columns = ['Удаленный признак', 'accuracy@5', 'Время (сек.)']
result = pd.DataFrame(res_list, columns=columns)
display(result)

del df_train_drop, df_base_drop

In [22]:
# Удалю рассмотренные признаки, которые ведут себя аномально, оставив колонки №№ 6 и 70. При их удалении метрика меняется в худшую сторону. 
for df in [df_train, df_base, df_test]:
    df.drop(['21', '25', '33', '59', '44', '65'], axis=1, inplace=True)

### Промежуточный вывод. 
Данные не имеют пропусков и дубликатов. 
Признаки 6, 21, 25, 33, 44, 59, 65, 70 имеют ненормальное распределение. Остальные признаки имеют нормальное или близкое к нормальному распределение.
Корреляция признаков отсутствует.
Данные имеют выбросы. 

[К оглавлению](#section_0)
<a id='section_3'></a>
## Подготовка и выбор моделей  

In [23]:
# Подберу скалер для трансформации данных. 
all_scaler = [
    MinMaxScaler(), MaxAbsScaler(), PowerTransformer(),
    QuantileTransformer(), RobustScaler()
]

res_list = []

for scal in tqdm(all_scaler):
    scaler = scal
    base_transform  = scaler.fit_transform(df_base)
    train_transform = scaler.transform(df_train)
    index = faiss.IndexFlatL2(base_transform.shape[1])

    acc, time_ = accuracy_n(train_transform, base_transform, idx=index)
    res_list.append([scal, acc, time_])

columns = ['Scaler', 'accuracy@5', 'Время (сек.)']
result = pd.DataFrame(res_list, columns=columns)
result.sort_values(by='accuracy@5', ascending=False)

RobustScaler 71.9450 
PowerTransformer 71.879
StandardScaler 71.876
QuantileTransformer 71.446
MinMaxScaler 71.01
MaxAbsScaler 70.16

Наилучшие показатели метрики у RobustScaler. Применю его к данным. 

In [24]:
scaler = RobustScaler()

base_transform  = scaler.fit_transform(df_base)
train_transform = scaler.transform(df_train)
test_transform = scaler.transform(df_test)

In [25]:
# Выбор числа кластеров по методу "локтя"

inertia = []
n_clusters = [5, 10, 50, 100, 200, 400, 600]

for i in tqdm(n_clusters):
    kmeans = KMeans(n_clusters=i, 
                    random_state=RANDOM_STATE, 
                    init='k-means++',
                    n_init='auto',
                    algorithm='elkan',
                    ).fit(base_transform)
    inertia.append(np.sqrt(kmeans.inertia_))

plt.plot(n_clusters, inertia)
plt.figure(figsize=(30, 20))
plt.xlabel('$k$')
plt.ylabel('$J(C_k)$');

 Заметен излом кривой на 200 кластерах. 

In [26]:
# Вычисление кол-ва кластеров

def get_clusters_coeff(df:pd.DataFrame, n_clusters:list):
    if isinstance(n_clusters, int):
        n_clusters = [n_clusters]

    for i in tqdm(n_clusters):
        cluster_k = KMeans(n_clusters=i,
                           n_init='auto',
                           init='k-means++',
                           random_state=RANDOM_STATE).fit(df)
        # Разделим на i кластеров
        db_score = davies_bouldin_score(df, cluster_k.labels_)
        print(f'{i} кластеров: {round(db_score, 4)}')

In [27]:
%%time
get_clusters_coeff(df=base_transform, 
                   n_clusters=[5, 10, 25, 50, 100, 200, 400, 600])

И поиск кластеров коэффициентом схожести Девида-Боулдина также показвает на 200 кластеров. 
<br>
### Рассмотрение различных индексов FAISS

In [28]:
dimension = base_transform.shape[1]

In [29]:
res_list = []
nlists = 200
quantizer = faiss.IndexFlatL2(dimension)

ind = {'IndexFlatL2': faiss.IndexFlatL2(dimension),
       'IndexIVFFlat': faiss.IndexIVFFlat(quantizer, dimension, nlists),
       'IndexHNSWFlat': faiss.IndexHNSWFlat(dimension, K),
       'IndexIVFPQ': faiss.IndexIVFPQ(quantizer, dimension, nlists, 11, 8),
       'IndexPQ': faiss.IndexPQ(dimension, 11, 8),
       'IndexLSH': faiss.IndexLSH(dimension, 8),
       'IndexFlatIP': faiss.IndexFlatIP(dimension),
       }

for k, v in tqdm(ind.items()):
    acc, time_ = accuracy_n(train_transform, base_transform, idx=v)
    res_list.append([k, acc, time_])

columns = ['Index', 'accuracy@5', 'Время (сек.)']
result = pd.DataFrame(res_list, columns=columns)
result.sort_values(by='accuracy@5', ascending=False)

Наилучшие показатели у индексов ***IndexIVFFlat*** и ***IndexFlatL2***, но при этом ***IndexFlatL2*** работает в 10 раз быстрее, а результат у него отличается всего на 2 тысячных.
При таких показателях выберу для дальнейшей работы ***IndexFlatL2***

[К оглавлению](#section_0)
<a id='section_4'></a>
## Уменьшение размерности  

Попробую уменьшить размерность и посмотрю на результаты. 

In [30]:
# Создаем экземпляр PCA
pca = PCA(n_components=None,
          whiten=True,
          random_state=RANDOM_STATE).fit(base_transform)

ipca = IncrementalPCA(n_components=None,
                      whiten=True,
                      batch_size=100).fit(base_transform)

# Строим график объясненной дисперсии
evr_ipca = ipca.explained_variance_ratio_
evr_pca = pca.explained_variance_ratio_

plt.plot(range(1, len(evr_ipca) + 1), evr_ipca, marker='o', color='darkorange')
plt.plot(range(1, len(evr_pca) + 1), evr_pca, marker='X', color='navy')

plt.xlabel("Число компонент")
plt.ylabel("Доля объясненной дисперсии")
plt.title("Метод локтя для выбора числа компонент")
plt.show()

In [31]:
pca = PCA(n_components=dimension,
          whiten=True,
          random_state=RANDOM_STATE).fit(base_transform)

acc, time_ = accuracy_n(
    pca.transform(train_transform), 
    pca.transform(base_transform),
    idx=faiss.IndexFlatL2(dimension)
)
print(f'accuracy@5 = {acc}, время выполнения = {time_} секунд.')

In [32]:
ipca = IncrementalPCA(n_components=dimension,
                      whiten=True,
                      batch_size=100).fit(base_transform)

acc, time_ = accuracy_n(
    ipca.transform(train_transform),
    ipca.transform(base_transform),
    idx=faiss.IndexFlatL2(dimension)
)
print(f'accuracy@5 = {acc}, время выполнения = {time_} секунд.')

In [33]:
svd = TruncatedSVD(n_components=dimension,
                   random_state=RANDOM_STATE).fit(base_transform)

acc, time_ = accuracy_n(
    svd.transform(train_transform),
    svd.transform(base_transform),
    idx=faiss.IndexFlatL2(dimension)
)
print(f'accuracy@5 = {acc}, время выполнения = {time_} секунд.')

Эксперименты с уменьшением размерности не дали улучшения метрики. 

[К оглавлению](#section_0)
<a id='section_5'></a>
## Проверка на Валидационной выборке

Проверю результат на валидационных данных. 

In [35]:
acc, time_ = accuracy_n(
    test_transform,
    base_transform,
    targets=ans['Expected'],
    idx=faiss.IndexFlatL2(dimension)
)
print(f'accuracy@5 = {acc}, время выполнения = {time_} секунд.')

accuracy@5 = 71.798, время выполнения = 239.22 секунд.


In [36]:
nlists = 200
quantizer = faiss.IndexFlatL2(dimension)

acc, time_ = accuracy_n(
    test_transform,
    base_transform,
    targets=ans['Expected'],
    idx=faiss.IndexIVFFlat(quantizer, dimension, nlists)
)
print(f'accuracy@5 = {acc}, время выполнения = {time_} секунд.')

accuracy@5 = 71.797, время выполнения = 2221.57 секунд.


[К оглавлению](#section_0)
<a id='section_6'></a>
## Итоговый вывод

Значение целевой метрики ***accuracy@5 = 71.798*** было получено с использованием модели **FAISS** с индексом `IndexFlatL2`.
Масштабирование было выполнено с применением *RobustScaler*. 
Признаки с ненормальным распределением были удалены, за исключением двух, удаление которых неблагоприятно влияло на метрику (6 и 70).
Попытка уменьшить размерность не привела к улучшению результата. 